In [1]:
import pandas as pd
import math

#pull reviews
reviews = pd.read_csv("reviews.csv")

#take train data...
num_test = math.floor(len(reviews) * .2)
num_train = len(reviews) - num_test

print(len(reviews))
print(num_test)
print(num_train)

2511
502
2009


In [2]:
test = reviews.iloc[:num_test]
train = reviews.iloc[num_test:]

print(len(test))
print(len(train))

502
2009


In [52]:
test_labels = test['rating'].to_frame()
test_text = test['text'].to_frame()

train_labels = train['rating'].to_frame()
train_text = train['text'].to_frame()

In [4]:
import nltk
from nltk.tokenize import WordPunctTokenizer

tk = WordPunctTokenizer()
NEG = 'NOT_'
negs = ['not', 'none', 'no', 'impossible', 'n\'t']
punct = [',', '.', '?', '!', ';', '-']

#preps data
def prep(text):
    data_list = []
    for i, data in text.iterrows():
        words = []
        review = data['text']
        listed = review.split()
        neggy = False
        for word in listed:
            w = word.lower()
            if neggy:
                if word in punct:
                    words.append(w)
                    neggy = False
                else:
                    words.append(NEG + w)
            else:
                words.append(w)
                if w in negs:
                    if neggy:
                        neggy = False
                    else:
                        neggy = True
        data_list.append(' '.join(words))
    return data_list
 
train_list = prep(train_text)
tokens = tk.tokenize(' '.join(train_list))
freq = nltk.FreqDist(tokens)
vocab = {'<unk>':0}
i = 1
for item in freq.keys():
    if item not in vocab:
        if freq[item] > 2:
            vocab[item] = i
            i += 1
print(len(vocab))

11768


In [42]:
import numpy as np

def make_matrix(text_list):
    n = len(text_list)
    X = np.zeros((n, len(vocab)))
    for i in range(n):
        review = tk.tokenize(text_list[i])
        for word in review:
            if word in vocab:
                X[i][vocab[word]] += 1
            else:
                X[i][0] += 1
    return X

train_matrix = make_matrix(train_list)

print("train feature matrix shape: ({}, {})".format(len(train_matrix), len(train_matrix[0])))
print("train label array shape: ({},)".format(len(train_labels)))

train feature matrix shape: (2009, 11768)
train label array shape: (2009,)


In [43]:
#clean test data

test_list = prep(test_text)
test_matrix = make_matrix(test_list)

print("test feature matrix shape: ({}, {})".format(len(test_matrix), len(test_matrix[0])))
print("test label array shape: ({},)".format(len(test_labels)))

test feature matrix shape: (502, 11768)
test label array shape: (502,)


In [46]:
def clean_labels(y):
    for i, val in enumerate(y):
        if val > 3:
            y[i] = 1
        else:
            y[i] = -1
    return y

X_train = train_matrix
y_train = clean_labels(np.ravel(train_labels.to_numpy()))

X_test = test_matrix
y_test = clean_labels(np.ravel(test_labels.to_numpy()))

In [31]:
y_train.shape

(2009,)

In [ ]:
import time

In [48]:
#model for pos/negative review

from numpy import arange
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

model = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)

model.fit(X_train, y_train)

print(model.alpha_)

/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.36927251401766625, tolerance: 0.22715790929203458
  model = cd_fast.enet_coordinate_descent(
/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.519818598369942, tolerance: 0.22920353982300853
  model = cd_fast.enet_coordinate_descent(
/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

0.01


In [49]:
y_hat = model.predict(X_train)

def reg_score(y_hat):
    for i, val in enumerate(y_hat):
        if val < 0:
            y_hat[i] = -1
        else:
            y_hat[i] = 1
    return y_hat

y_hat = reg_score(y_hat)
loss = np.linalg.norm(y_hat - y_train)
print(loss)

38.8329756778952


In [50]:
y_hat_test = model.predict(X_test)
y_hat_test = reg_score(y_hat_test)
loss = np.linalg.norm(y_hat_test - y_test)
print(loss)

19.697715603592208


In [55]:
#model for number of stars review

test_labels = test['rating'].to_frame()
train_labels = train['rating'].to_frame()

X_train = train_matrix
y_train_nums = np.ravel(train_labels.to_numpy())

X_test = test_matrix
y_test_nums = np.ravel(test_labels.to_numpy())

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

nums_model = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)

nums_model.fit(X_train, y_train_nums)

print(nums_model.alpha_)

[5. 2. 5. 1. 3. 3. 4. 5. 5. 3. 4. 4. 4. 4. 4. 1. 3. 5. 3. 5. 5. 4. 5. 3.
 4. 4. 5. 5. 5. 2. 3. 5. 5. 4. 1. 5. 4. 5. 4. 2. 5. 4. 4. 5. 5. 5. 4. 5.
 3. 4. 2. 4. 4. 4. 5. 4. 4. 5. 5. 2. 5. 5. 5. 5. 4. 4. 5. 4. 3. 1. 2. 3.
 3. 3. 4. 5. 5. 4. 4. 5. 3. 4. 3. 5. 4. 1. 1. 5. 4. 3. 3. 4. 2. 3. 4. 5.
 3. 2. 1. 5. 5. 3. 5. 4. 4. 3. 5. 5. 5. 5. 3. 3. 2. 1. 3. 4. 1. 3. 2. 5.
 2. 2. 3. 4. 4. 5. 5. 3. 5. 4. 5. 3. 5. 5. 4. 2. 4. 4. 5. 4. 4. 2. 2. 3.
 4. 4. 4. 5. 2. 3. 4. 4. 4. 3. 4. 2. 4. 5. 4. 4. 5. 5. 5. 5. 4. 3. 4. 3.
 5. 5. 5. 5. 4. 3. 2. 5. 3. 5. 5. 5. 5. 5. 5. 4. 4. 1. 4. 4. 5. 5. 5. 4.
 4. 2. 4. 5. 5. 4. 4. 5. 5. 4. 4. 5. 2. 4. 5. 4. 2. 5. 5. 5. 4. 4. 2. 4.
 1. 5. 5. 5. 5. 5. 5. 1. 5. 5. 4. 4. 5. 4. 5. 5. 2. 2. 4. 5. 4. 1. 5. 1.
 4. 5. 5. 5. 5. 5. 5. 3. 4. 5. 4. 2. 3. 2. 4. 4. 5. 5. 3. 1. 5. 4. 4. 4.
 5. 3. 5. 4. 4. 4. 4. 4. 5. 5. 5. 5. 5. 5. 5. 5. 4. 5. 5. 4. 4. 4. 5. 5.
 5. 4. 5. 5. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 4. 5. 4. 5.
 5. 1. 5. 3. 4. 5. 4. 5. 4. 4. 4. 4. 5. 4. 4. 5. 5.

/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.3647659417479936, tolerance: 0.13519911504424814
  model = cd_fast.enet_coordinate_descent(
/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.25198875422427136, tolerance: 0.13619800884955724
  model = cd_fast.enet_coordinate_descent(
/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning

0.01


/Users/rayfregly/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3002482102783688, tolerance: 0.25375311100049797
  model = cd_fast.enet_coordinate_descent(


In [57]:
y_hat_nums = nums_model.predict(X_train)

loss = np.linalg.norm(y_hat_nums - y_train_nums)
print(loss)

y_hat_test_nums = nums_model.predict(X_test)
loss = np.linalg.norm(y_hat_test_nums - y_test_nums)
print(loss)

39.94947021280382
22.013886501596104
